In [1]:
from rdflib import *
from rdflib import URIRef

In [2]:
spoonR = Graph()

In [3]:
spoonR.parse("/Users/anita/Desktop/ERAS/ontology/kg.ttl", format="ttl")

<Graph identifier=N86e01fc2a56443909ece5f92f0a79017 (<class 'rdflib.graph.Graph'>)>

In [112]:
#What are the topics covered in each album?
q1=   """ 
SELECT ?album ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :hasTopic ?topic .
}
"""

#5.	Which is the frequency of the topics across the albums?
q2= """
SELECT ?topic (COUNT(?album) AS ?count)
WHERE {
  ?album rdf:type :Album .
  ?album :hasTopic ?topic .
}
GROUP BY ?topic
ORDER BY ?topic
    """

#How do topics evolve over the years?
q3 = """
SELECT DISTINCT ?releaseYear ?topic
WHERE {
  ?song rdf:type :Song .
  ?song :has_release_year ?releaseYear .
  ?song :hasTopic ?topic .
}
ORDER BY ?releaseYear
"""

#What topics are related to the different genres?
q4= """
SELECT DISTINCT ?genre ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :hasGenre ?genre .
  ?album :hasTopic ?topic .
}
"""

q5= """
SELECT ?genre ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :hasGenre ?genre .
  ?album :hasTopic ?topic .
}
GROUP BY ?genre ?topic
ORDER BY ?genre

    """

#Which is the distribution of album across the genres? genres?
q6="""
SELECT ?genre (COUNT(?album) AS ?count)
WHERE {
  ?album rdf:type :Album .
  ?album :has_release_year ?year .
  ?album :hasGenre ?genre .
}
GROUP BY ?genre
ORDER BY ?genre
"""

#Which topics are associated with the labels?
q7="""
SELECT ?label ?topic
WHERE {
  { SELECT ?label ?topic
    WHERE {
      ?album rdf:type :Album .
      ?album :belongsToLabel ?label .
      ?album :hasTopic ?topic .
      FILTER (?label = :Big_Machine)
    }
  }
  UNION
  { SELECT ?label ?topic
    WHERE {
      ?album rdf:type :Album .
      ?album :belongsToLabel ?label .
      ?album :hasTopic ?topic .
      FILTER (?label = :Republic)
    }
  }
  UNION
  { SELECT ?label ?topic
    WHERE {
      ?album rdf:type :Album .
      ?album :belongsToLabel ?label .
      ?album :hasTopic ?topic .
      FILTER (?label = :Taylor_Swift_Productions)
    }
  }
}

"""


#LOVER AND REPUTATION
q8="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "reputation")
}

"""

q9 = """
SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "Lover")
}


"""

q10 = """
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "reputation")
}

"""

q11 = """
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "Lover")
}

"""

#FOKLORE AND EVERMORE
q12="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "folklore")
}

"""


q13="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "evermore")
}

"""


q14="""
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "folklore")
}

"""

q15="""
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "evermore")
}

"""

#ALL FEATURED SONGS

q16= """
SELECT ?songTitle ?featuredArtist
WHERE {
  ?song rdf:type :Song .
  ?song :has_title ?songTitle .
  ?song :hasFeaturedArtist ?featuredArtist .
}

"""


#Longest album
q17="""
SELECT ?albumTitle ?duration
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :has_duration ?duration .
}
ORDER BY DESC(?duration)
LIMIT 1"""

#shortest album

q18="""
SELECT ?albumTitle ?duration
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :has_duration ?duration .
}
ORDER BY ASC(?duration)
LIMIT 1

"""

#midnight and the tortured poets department

q19="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "Midnights")
}

"""


q20="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "The Tortured Poets Department")
}

"""


q21="""
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "Midnights")
}

"""

q22="""
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "The Tortured Poets Department")
}

"""

q23="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "Taylor Swift")
}

"""


q24="""

SELECT ?albumTitle ?topic
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasTopic ?topic .
  FILTER (str(?albumTitle) = "The Tortured Poets Department")
}

"""


q25="""
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "Taylor Swift")
}

"""

q26="""
SELECT ?albumTitle ?genre
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :hasGenre ?genre .
  FILTER (str(?albumTitle) = "The Tortured Poets Department")
}

"""



queries = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, q13, q14, q15, q16, q17, q18, q19, q20, q21, q22, q23, q24, q25,q26, q27]

# Function to extract local name from a URIRef
def get_local_name(uri):
    return uri.split('#')[-1]

for idx, q in enumerate(queries):
    print("Results for query " + str(idx + 1) + ":")
    try:
        results = spoonR.query(q)
        if results:
            for result in results:
                album, topic = result
                album_local = get_local_name(str(album))
                topic_local = get_local_name(str(topic))
                print(f"result: {album_local}, result: {topic_local}")
        else:
            print("No results found.")
    except Exception as e:
        print(f"An error occurred: {e}")

Results for query 1:
result: 1989, result: http://www.semanticweb.org/ERAS-ontology/desire
result: 1989, result: http://www.semanticweb.org/ERAS-ontology/enjoying_life
result: 1989, result: http://www.semanticweb.org/ERAS-ontology/moving_on
result: 1989, result: http://www.semanticweb.org/ERAS-ontology/relationships
result: 1989, result: http://www.semanticweb.org/ERAS-ontology/romantic_vibes
result: 1989, result: http://www.semanticweb.org/ERAS-ontology/turmoil
result: http://www.semanticweb.org/ERAS-ontology/Debut, result: http://www.semanticweb.org/ERAS-ontology/appreciation
result: http://www.semanticweb.org/ERAS-ontology/Debut, result: http://www.semanticweb.org/ERAS-ontology/breakup
result: http://www.semanticweb.org/ERAS-ontology/Debut, result: http://www.semanticweb.org/ERAS-ontology/expression
result: http://www.semanticweb.org/ERAS-ontology/Debut, result: http://www.semanticweb.org/ERAS-ontology/intimacy
result: http://www.semanticweb.org/ERAS-ontology/Debut, result: http://w